In [1]:
import os
# Find the latest version of spark 3.1  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.1.1'
spark_version = 'spark-3.1.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

import pandas as pd

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Get:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release [564 B]
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release.gpg [833 B]
Get:11 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]


In [3]:
#download a Postgres driver that will allow Spark to interact with Postgres:

!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2021-03-07 15:17:25--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar.1’

postgresql-42.2.16. 100%[===================>] 979.38K  --.-KB/s    in 0.09s   

2021-03-07 15:17:26 (11.1 MB/s) - ‘postgresql-42.2.16.jar.1’ saved [1002883/1002883]



In [28]:
#start a Spark session with an additional option that adds the driver to Spark:

from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [29]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url ="https://emmp-finalproject-bucket.s3.amazonaws.com/df_econ2.csv"
spark.sparkContext.addFile(url)
economic_data_df = spark.read.csv(SparkFiles.get("df_econ2.csv"), sep=",", header=True, inferSchema=True)

In [30]:
# Show DataFrame
economic_data_df.show()

+------+------------+------------+------------+--------+---------+--------+
|  Date|   inflation|    S&Pyield|unemployment|UMCSENT4|homeindex|UMCSENT6|
+------+------------+------------+------------+--------+---------+--------+
|200001|         0.0|-0.610842585|         4.0|   111.3|   0.0646|   111.3|
|200002| 0.068801898|-0.241296999|         4.1|   107.1|   0.0646|   107.1|
|200003| 0.096717122| 1.160637939|         4.0|   109.2|   0.0646|   109.2|
|200004| 0.009360374|-0.369549067|         3.8|   110.7|   0.0666|   110.7|
|200005| 0.009353079|-0.262980606|         4.0|   106.4|   0.0666|   106.4|
|200006| 0.067757009| 0.287202595|         4.0|   108.3|   0.0666|   108.3|
|200007| 0.023233301|-0.196095315|         4.0|   107.3|    0.067|   107.3|
|200008| 0.002318841| 0.728389262|         4.1|   106.8|    0.067|   106.8|
|200009|   0.0625966|-0.641795697|         3.9|   105.8|    0.067|   105.8|
|200010| 0.020757255| -0.05939383|         3.9|   107.6|   0.0695|   107.6|
|200011| 0.0

In [31]:
# Read in data from S3 Buckets - Media Data

url ="https://emmp-finalproject-bucket.s3.amazonaws.com/Movies-Cleaned-Final2.csv"
spark.sparkContext.addFile(url)
media_df = spark.read.csv(SparkFiles.get("Movies-Cleaned-Final2.csv"), sep=",", header=True, inferSchema=True)

# Show DataFrame
media_df.show()

+------+-----------+---------+
|  Date|Total Gross|    Genre|
+------+-----------+---------+
|200601|   64724295|   Action|
|200602|    8357325|   Action|
|200603|  125928240|   Action|
|200604|       2220|   Action|
|200605|  368533435|   Action|
|200606|   65454326|   Action|
|200607|   63520641|   Action|
|200608|       null|   Action|
|200609|   94964871|   Action|
|200610|   21727955|   Action|
|200611|  170008586|   Action|
|200612|  114811324|   Action|
|200601|   98665890|Adventure|
|200602|  230058104|Adventure|
|200603|  195330621|Adventure|
|200604|  110323204|Adventure|
|200605|  223811794|Adventure|
|200606|  472629729|Adventure|
|200607|  525119357|Adventure|
|200608|   84768328|Adventure|
+------+-----------+---------+
only showing top 20 rows



In [32]:
#Renaming total gross to remove space to make it compatible with SQL database 
# Removing nulls 

media_df = media_df.withColumnRenamed("Total Gross", "Gross")
media_df = media_df.dropna()
media_df.show()

+------+---------+---------+
|  Date|    Gross|    Genre|
+------+---------+---------+
|200601| 64724295|   Action|
|200602|  8357325|   Action|
|200603|125928240|   Action|
|200604|     2220|   Action|
|200605|368533435|   Action|
|200606| 65454326|   Action|
|200607| 63520641|   Action|
|200609| 94964871|   Action|
|200610| 21727955|   Action|
|200611|170008586|   Action|
|200612|114811324|   Action|
|200601| 98665890|Adventure|
|200602|230058104|Adventure|
|200603|195330621|Adventure|
|200604|110323204|Adventure|
|200605|223811794|Adventure|
|200606|472629729|Adventure|
|200607|525119357|Adventure|
|200608| 84768328|Adventure|
|200609|100385675|Adventure|
+------+---------+---------+
only showing top 20 rows



In [34]:
#Renaming media date column
media_df = media_df.withColumnRenamed("Date", "mDate")
media_df.show()

+------+---------+---------+
| mDate|    Gross|    Genre|
+------+---------+---------+
|200601| 64724295|   Action|
|200602|  8357325|   Action|
|200603|125928240|   Action|
|200604|     2220|   Action|
|200605|368533435|   Action|
|200606| 65454326|   Action|
|200607| 63520641|   Action|
|200609| 94964871|   Action|
|200610| 21727955|   Action|
|200611|170008586|   Action|
|200612|114811324|   Action|
|200601| 98665890|Adventure|
|200602|230058104|Adventure|
|200603|195330621|Adventure|
|200604|110323204|Adventure|
|200605|223811794|Adventure|
|200606|472629729|Adventure|
|200607|525119357|Adventure|
|200608| 84768328|Adventure|
|200609|100385675|Adventure|
+------+---------+---------+
only showing top 20 rows



In [35]:
#Create economics DataFrames that match what is in the AWS RDS database.

#create economics DF to match economics table
clean_economics_df = economic_data_df.select(['Date', 'inflation', 'S&Pyield', 'unemployment', 'UMCSENT4', 'homeindex'])
clean_economics_df.show()

+------+------------+------------+------------+--------+---------+
|  Date|   inflation|    S&Pyield|unemployment|UMCSENT4|homeindex|
+------+------------+------------+------------+--------+---------+
|200001|         0.0|-0.610842585|         4.0|   111.3|   0.0646|
|200002| 0.068801898|-0.241296999|         4.1|   107.1|   0.0646|
|200003| 0.096717122| 1.160637939|         4.0|   109.2|   0.0646|
|200004| 0.009360374|-0.369549067|         3.8|   110.7|   0.0666|
|200005| 0.009353079|-0.262980606|         4.0|   106.4|   0.0666|
|200006| 0.067757009| 0.287202595|         4.0|   108.3|   0.0666|
|200007| 0.023233301|-0.196095315|         4.0|   107.3|    0.067|
|200008| 0.002318841| 0.728389262|         4.1|   106.8|    0.067|
|200009|   0.0625966|-0.641795697|         3.9|   105.8|    0.067|
|200010| 0.020757255| -0.05939383|         3.9|   107.6|   0.0695|
|200011| 0.006907137| -0.96082332|         3.9|    98.4|   0.0695|
|200012|-0.009204219| 0.048641346|         3.9|    94.7|   0.0

In [36]:
#Removing NA

clean_economics_df = clean_economics_df.dropna()
clean_economics_df.show()

+------+------------+------------+------------+--------+---------+
|  Date|   inflation|    S&Pyield|unemployment|UMCSENT4|homeindex|
+------+------------+------------+------------+--------+---------+
|200001|         0.0|-0.610842585|         4.0|   111.3|   0.0646|
|200002| 0.068801898|-0.241296999|         4.1|   107.1|   0.0646|
|200003| 0.096717122| 1.160637939|         4.0|   109.2|   0.0646|
|200004| 0.009360374|-0.369549067|         3.8|   110.7|   0.0666|
|200005| 0.009353079|-0.262980606|         4.0|   106.4|   0.0666|
|200006| 0.067757009| 0.287202595|         4.0|   108.3|   0.0666|
|200007| 0.023233301|-0.196095315|         4.0|   107.3|    0.067|
|200008| 0.002318841| 0.728389262|         4.1|   106.8|    0.067|
|200009|   0.0625966|-0.641795697|         3.9|   105.8|    0.067|
|200010| 0.020757255| -0.05939383|         3.9|   107.6|   0.0695|
|200011| 0.006907137| -0.96082332|         3.9|    98.4|   0.0695|
|200012|-0.009204219| 0.048641346|         3.9|    94.7|   0.0

In [37]:
#Renaming S&P to remove ampere sign to make compatible with SQL database 

clean_economics_df = clean_economics_df.withColumnRenamed("S&Pyield", "SPyield")
clean_economics_df.show()

+------+------------+------------+------------+--------+---------+
|  Date|   inflation|     SPyield|unemployment|UMCSENT4|homeindex|
+------+------------+------------+------------+--------+---------+
|200001|         0.0|-0.610842585|         4.0|   111.3|   0.0646|
|200002| 0.068801898|-0.241296999|         4.1|   107.1|   0.0646|
|200003| 0.096717122| 1.160637939|         4.0|   109.2|   0.0646|
|200004| 0.009360374|-0.369549067|         3.8|   110.7|   0.0666|
|200005| 0.009353079|-0.262980606|         4.0|   106.4|   0.0666|
|200006| 0.067757009| 0.287202595|         4.0|   108.3|   0.0666|
|200007| 0.023233301|-0.196095315|         4.0|   107.3|    0.067|
|200008| 0.002318841| 0.728389262|         4.1|   106.8|    0.067|
|200009|   0.0625966|-0.641795697|         3.9|   105.8|    0.067|
|200010| 0.020757255| -0.05939383|         3.9|   107.6|   0.0695|
|200011| 0.006907137| -0.96082332|         3.9|    98.4|   0.0695|
|200012|-0.009204219| 0.048641346|         3.9|    94.7|   0.0

In [39]:
#Renaming Date column 
clean_economics_df = clean_economics_df.withColumnRenamed("Date", "eDate")
clean_economics_df.show()

+------+------------+------------+------------+--------+---------+
| eDate|   inflation|     SPyield|unemployment|UMCSENT4|homeindex|
+------+------------+------------+------------+--------+---------+
|200001|         0.0|-0.610842585|         4.0|   111.3|   0.0646|
|200002| 0.068801898|-0.241296999|         4.1|   107.1|   0.0646|
|200003| 0.096717122| 1.160637939|         4.0|   109.2|   0.0646|
|200004| 0.009360374|-0.369549067|         3.8|   110.7|   0.0666|
|200005| 0.009353079|-0.262980606|         4.0|   106.4|   0.0666|
|200006| 0.067757009| 0.287202595|         4.0|   108.3|   0.0666|
|200007| 0.023233301|-0.196095315|         4.0|   107.3|    0.067|
|200008| 0.002318841| 0.728389262|         4.1|   106.8|    0.067|
|200009|   0.0625966|-0.641795697|         3.9|   105.8|    0.067|
|200010| 0.020757255| -0.05939383|         3.9|   107.6|   0.0695|
|200011| 0.006907137| -0.96082332|         3.9|    98.4|   0.0695|
|200012|-0.009204219| 0.048641346|         3.9|    94.7|   0.0

In [49]:
# Load - transformed raw data into our database.
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://dbfinal.cp1bf7yng3ct.us-east-1.rds.amazonaws.com:5432/final_project"
config = {"user":"postgres",
          "password": "Bitro123*",
          "driver":"org.postgresql.Driver"}

In [50]:
# DataFrame to economics table
clean_economics_df.write.jdbc(url=jdbc_url, table='economics', mode=mode, properties=config)

In [51]:
# DataFrame to media_info table
media_df.write.jdbc(url=jdbc_url, table='media_info', mode=mode, properties=config)

In [ ]:
# Join the tables on the database 

In [70]:
#Importing the economics & media data combined 

medianeconomics_table = spark.read.jdbc(url=jdbc_url, table="medianeconomics", properties=config)


In [71]:
medianeconomics_table.show()

+------+---------+-------+------------+--------+---------+---------+---------+
| edate|inflation|spyield|unemployment|umcsent4|homeindex|    genre|    gross|
+------+---------+-------+------------+--------+---------+---------+---------+
|200601|        0|      0|           5|      87|        0|   Action| 64724295|
|200602|        0|      0|           5|      89|        0|   Action|  8357325|
|200603|        0|      0|           5|      87|        0|   Action|125928240|
|200604|        0|      0|           5|      79|        0|   Action|     2220|
|200605|        0|      0|           5|      85|        0|   Action|368533435|
|200606|        0|      0|           5|      85|        0|   Action| 65454326|
|200607|        0|      0|           5|      82|        0|   Action| 63520641|
|200609|        0|      0|           4|      94|        0|   Action| 94964871|
|200610|        0|      0|           4|      92|        0|   Action| 21727955|
|200611|        0|      0|           4|      92|    

In [92]:
# Save file local folder, delimiter by default is ,
medianeconomics_table.write.format('csv').option('header',True).mode('overwrite').option('sep',',').save('file:///Mac_1/Downloads/output.csv')

#OR

#medianeconomics_table.repartition(1).write.format('com.databricks.spark.csv').save('file:///Mac_1/Downloads/output2.csv',header = 'true')